In [1]:
import pandas as pd

raw_df = pd.read_feather("../data/data.feather")
test_items_df = pd.read_csv("../data/test_items.csv",index_col=0)
question_data = pd.read_csv("../data/question_data.csv", sep=';', index_col=0)

In [2]:
test_item_qs = [item for item in test_items_df.index if item in raw_df.columns]

In [3]:
q_df = raw_df.drop(columns=test_item_qs)
q_df = q_df.filter(regex=r'^q\d+$')

In [4]:
# drop the lowest response q's
N_DROP = 2000
low_response = [col for col in question_data.sort_values('N').iloc[:N_DROP].index if col in q_df.columns]
q_df = q_df.drop(columns=low_response)

In [5]:
# sort by number of categories for easier manipulation later
sorted_num_levels = q_df.apply(lambda x: len(x.cat.categories)).sort_values()
q_df = q_df[sorted_num_levels.index]

In [6]:
# Get the number of unique levels per column
num_levels = q_df.nunique()

# Sort columns by unique levels
sorted_num_levels = num_levels.sort_values()

# Find the indices where the level count changes
level_counts = sorted_num_levels.values
diff = level_counts[1:] != level_counts[:-1]
border_indices = list(diff.nonzero()[0] + 1)  # +1 because diff is between elements

# For example, print the borders and corresponding levels
print("Border indices where level count changes:", border_indices)
print("Levels at borders:", level_counts[border_indices])

# You can use these indices to split the sorted columns into dfs as needed

Border indices where level count changes: [252, 392]
Levels at borders: [3 4]


In [7]:
import numpy as np

non_nan = ~q_df.isna()
non_nan_indices = np.flatnonzero(non_nan.values)

np.random.seed(0)
TEST_SIZE = 0.2
test_size = int(len(non_nan_indices) * TEST_SIZE)
test_mask_flat = np.random.choice(non_nan_indices, size=test_size, replace=False)

test_mask = np.zeros_like(q_df.values, dtype=bool)
test_mask.flat[test_mask_flat] = True

# mask some cells that serve as our test set
df_masked = q_df.mask(test_mask)


In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(sparse_output=True, handle_unknown='ignore'), q_df.columns),
    ],
    sparse_threshold=1.0
)

def transform_and_drop(df):
    X = preprocessor.fit_transform(df)

    # the sklearn onehot encoder doesn't have an option to not encode nans
    feature_names = preprocessor.get_feature_names_out()
    cols_to_keep = [i for i, name in enumerate(feature_names) if not name.endswith('_nan')]
    X = X[:, cols_to_keep]
    return X

X_combined = transform_and_drop(df_masked)

In [9]:
import numpy as np
from scipy import sparse

# Original mask (shape: n_users x n_original_questions)

# convert the original mask to a mask over the onehots
def expand_mask(mask):
    mask_expanded = []
    for col_idx, col in enumerate(q_df.columns):
        n_categories = len(q_df[col].cat.categories)
        # Repeat mask along a new axis (shape: n_users x n_categories)
        mask_repeated = np.repeat(mask[:, col_idx][:, np.newaxis], n_categories, axis=1)
        mask_expanded.append(mask_repeated)
        # sanity check
        assert mask_repeated.shape[0] == 68371

    # Stack horizontally (shape: n_users x n_encoded_features)
    mask_expanded = np.hstack(mask_expanded)
    return mask_expanded

# Convert to sparse matrix (if X_combined is sparse)
test_mask_expanded = expand_mask(test_mask)
test_mask_sparse = sparse.csr_matrix(test_mask_expanded)

original_mask = expand_mask(non_nan.values)

In [11]:
# sanity check

print(test_mask_sparse.shape, X_combined.shape)
X_combined[test_mask_sparse.nonzero()].any()

(68371, 1660) (68371, 1660)


False

### data cleaned, ready for naive bayes

In [12]:
pr_q_answered = df_masked.notna().mean()
pr_user_answered = df_masked.notna().mean(axis=1)

In [13]:
feature_names = preprocessor.get_feature_names_out()

In [14]:
feature_names = preprocessor.get_feature_names_out()
kept_features = [item[8:].split('_') for item in feature_names if not item.endswith('_nan')]

In [15]:
# demonstration of how the naive imputation model works

for user, feature in zip(*test_mask_sparse.nonzero()):
    question, option = kept_features[feature]
    print(user, question, option)
    print("pr(user answers)=", pr_user_answered[user])
    print("pr(question gets answered)=", pr_q_answered[question])
    print("pr(option i selected | question K gets answered)=", df_masked[question].value_counts(normalize=True)[option])
    print("take the prodcut of these")
    break

0 q170849 No. Why spoil the mystery?
pr(user answers)= 0.3177083333333333
pr(question gets answered)= 0.46557751093299793
pr(option i selected | question K gets answered)= 0.6508859009801458
take the prodcut of these


In [16]:
questions = np.array([q for q, o in kept_features])
options   = np.array([o for q, o in kept_features])

option_probs = {
    q: df_masked[q].value_counts(normalize=True).reindex(q_df[q].cat.categories, fill_value=0)
    for q in q_df.columns
}
option_probs_df = pd.DataFrame(option_probs).T  # index: question, columns: option. super redundant but helps vectorize the operations below

users_idx, features_idx = test_mask_sparse.nonzero()

q_for_masked = questions[features_idx]
o_for_masked = options[features_idx]

pr_user_vals = pr_user_answered.values[users_idx]
pr_q_vals    = pr_q_answered[q_for_masked].values
pr_option_vals = option_probs_df.values[
    option_probs_df.index.get_indexer(q_for_masked),
    option_probs_df.columns.get_indexer(o_for_masked)
]

# Vectorized naive bayes
naive_imputed_values = pr_user_vals * pr_q_vals * pr_option_vals


In [17]:
# compare hidden test values to real values
X_not_masked = transform_and_drop(q_df)

# check:
X_not_masked.shape == X_combined.shape

True

In [18]:
# MSE
mse_naive = (np.asarray(X_not_masked[test_mask_sparse] - naive_imputed_values).ravel()**2).mean()
mse_naive

0.27301140033485116

In [19]:
# this is the one that Yoram explained to me, but I think the one above makes more sense.

row_mean = X_combined.mean(axis=1).ravel()
col_mean = X_combined.mean(axis=0).ravel()

naive_imputation2 = np.asarray(row_mean).ravel()[users_idx] * np.asarray(col_mean).ravel()[features_idx]

mse_naive2 = (np.asarray(X_not_masked[test_mask_sparse] - naive_imputation2).ravel()**2).mean()
mse_naive2

0.3185008577329666

### imputing values randomly

In [20]:
# random
mse_random = (np.asarray(X_not_masked[test_mask_sparse] - np.random.uniform(size=naive_imputed_values.shape)).ravel() **2).mean()
mse_random

0.3334107803502178

In [21]:
# impute as all zeros
mse_zero = (np.asarray(X_not_masked[test_mask_sparse] - np.zeros_like(naive_imputed_values)).ravel() **2).mean()
mse_zero

0.346698171916797

### imputing as modal answer

In [22]:
def get_mode(series):
    return series.mode().iloc[0]

df_modal_imputed = df_masked.apply(lambda col: col.fillna(get_mode(col)))
X_modal = transform_and_drop(df_modal_imputed)

In [23]:
# MSE
mse_modal = (np.asarray(X_not_masked[test_mask_sparse] - X_modal[test_mask_sparse]).ravel() **2).mean()
mse_modal

0.254023334054584

### low rank approximation method!

In [51]:
import torch
import torch.optim as optim
import torch.nn.functional as F

# Convert sparse matrices to PyTorch tensors
X_combined_tensor = torch.tensor(X_combined.toarray(), dtype=torch.float32)
test_mask_tensor = torch.tensor(test_mask_sparse.toarray(), dtype=torch.bool)
train_mask_tensor = torch.tensor(original_mask, dtype=torch.bool) & ~test_mask_tensor

# Hyperparameters
rank = 10  # Rank of the approximation (adjust as needed)
learning_rate = 0.1
epochs = 5

# Initialize low-rank matrices B and C
n_users, n_features = X_combined.shape
B = torch.randn(n_users, rank, requires_grad=True)
C = torch.randn(rank, n_features, requires_grad=True)
X_hat = torch.mm(B, C)

# Optimizer
optimizer = optim.Adam([B, C], lr=learning_rate)

# @torch.compile
def split_and_softmax(X):
    split_sizes = [border_indices[0]] + \
                [border_indices[i+1] - border_indices[i] for i in range(len(border_indices)-1)] + \
                [q_df.shape[1] - border_indices[-1]]

    split_sizes = [item * (i+2) for i, item in enumerate(split_sizes)]
    split_tensors = torch.split(X, split_sizes, dim=1)

    processed_tensors = []
    for i, tensor in enumerate(split_tensors):
        options_per_question = 2 + i  # 2 for first split, 3 for second, 4 for third
        n_questions = tensor.shape[1] // options_per_question
        
        # Reshape to (n_users, n_questions, options_per_question)
        reshaped = tensor.view(-1, n_questions, options_per_question)
        
        # Apply softmax along the last dimension
        softmaxed = F.softmax(reshaped, dim=-1)
        processed_tensors.append(softmaxed)
    
    flattened_tensors = [tensor.view(tensor.shape[0], -1) for tensor in processed_tensors]
    X_softmaxed = torch.cat(flattened_tensors, dim=1)
    return X_softmaxed

# Training loop
# @torch.compile
def train_loop():
    for epoch in range(epochs):
        optimizer.zero_grad()
        
        # Reconstruct X_hat = B @ C
        X_hat = torch.mm(B, C)
        X_hat = split_and_softmax(X_hat)

        # Compute loss only on observed entries (not masked for testing)
        loss = torch.mean((X_hat[train_mask_tensor] - X_combined_tensor[train_mask_tensor]) ** 2)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        print(f"Epoch {epoch}, Loss: {loss.item()}")
    return X_hat

X_hat = train_loop()


Epoch 0, Loss: 0.3477330505847931
Epoch 1, Loss: 0.32999297976493835
Epoch 2, Loss: 0.3125763535499573
Epoch 3, Loss: 0.295748770236969
Epoch 4, Loss: 0.2797464430332184


In [52]:

# After training, use X_hat to impute missing values
X_imputed = X_hat.detach().numpy()

# Evaluate on test set
test_values = np.asarray(X_not_masked[test_mask_sparse])
imputed_test_values = X_imputed[test_mask_sparse.toarray()]

mse_low_rank = ((test_values - imputed_test_values) ** 2).mean()
print(f"Low-rank MSE: {mse_low_rank}")

Low-rank MSE: 0.2890086825278953


In [42]:
original_mask

array([[False, False,  True, ...,  True,  True,  True],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [44]:
X_combined_tensor

tensor([[0., 0., 1.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [48]:
X_hat

tensor([[0.4497, 0.5503, 0.4998,  ..., 0.2506, 0.2327, 0.2524],
        [0.5028, 0.4972, 0.5000,  ..., 0.2499, 0.2510, 0.2499],
        [0.2171, 0.7829, 0.4984,  ..., 0.2456, 0.1531, 0.2567],
        ...,
        [0.7103, 0.2897, 0.5011,  ..., 0.2409, 0.3353, 0.2336],
        [0.8015, 0.1985, 0.5017,  ..., 0.2316, 0.3873, 0.2208],
        [0.9088, 0.0912, 0.5028,  ..., 0.2082, 0.4858, 0.1924]],
       grad_fn=<CatBackward0>)